In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('marketing_campaign.csv')

In [6]:
df

,Unnamed: 0,MarketSize,LocationID,Promotion,week,SalesInThousands,StoreAge
0,0,Medium,1,3,1,40.476,<=10 years
1,1,Medium,1,3,2,42.804,<=10 years
2,2,Medium,1,3,3,34.836,<=10 years
3,3,Medium,1,3,4,47.100,<=10 years
4,4,Medium,2,2,1,27.810,<=10 years
...,...,...,...,...,...,...,...
543,543,Large,919,1,4,64.340,<=10 years
544,544,Large,920,2,1,50.200,>10 years
545,545,Large,920,2,2,45.750,>10 years
546,546,Large,920,2,3,44.290,>10 years


# 1. What is the median sales (SalesInThousands)?

In [7]:
np.round(df['SalesInThousands'].median(),2)

53.63

# Answer: 53.63

# 2. Compute median sales in markets of different size (MarketSize). Markets of which size have the smallest median sales?

In [14]:
df.groupby('MarketSize').median()['SalesInThousands'].sort_values(ascending=True)

MarketSize
Medium    47.425
Small     61.420
Large     75.450
Name: SalesInThousands, dtype: float64

# Answer: Medium

# 3. To select best promotion strategy marketing team ran three promotion strategies (Promotion) for 4 weeks (week). Is there a more than 5% difference between average sales in Large markets across 4 weeks ? 

In [86]:
sales_mean = df[df.MarketSize=='Large'][['SalesInThousands']].mean().item()
sales_max = df[df.MarketSize=='Large'].groupby(['Promotion'])[['SalesInThousands']].mean().max().item()

In [87]:
round(100*(sales_max / sales_mean - 1), 2)

17.42

# Answer: yes

# 4. Which market size are the most important when selecting a promotion strategy? Compute total sales in markets of different size.

In [40]:
df.groupby('MarketSize').sum()['SalesInThousands'].sort_values(ascending=False)

MarketSize
Medium    15130.188
Large     12150.189
Small      4046.185
Name: SalesInThousands, dtype: float64

# Answer: Large - 15130, Medium - 12150, Small - 4046

# 5. Using bootstrap method sales team prepared 1000 samples of average sales in markets of Medium size (bootstrapped_means.csv). Help them estimate 99.5% confidence (two-sided) interval for average sales. (e.g. use `PERCENTILE` in google spreadsheets)

In [41]:
df_boots = pd.read_csv('bootstrapped_means.csv')
df_boots

,AverageSales,BootstrapID
0,46.894525,0
1,47.915681,1
2,48.206869,2
3,48.254650,3
4,47.017113,4
...,...,...
995,46.410175,995
996,46.739450,996
997,46.577144,997
998,46.739919,998


In [43]:
np.round(df_boots['AverageSales'].quantile(0.0025),2), np.round(df_boots['AverageSales'].quantile(0.9975),2)

(45.48, 49.02)

# Answer: [45.48, 49.02]

# 6. Compare sales in markets where different promotions were ran. Which promotion strategy showed the best results in terms of average sales?

In [47]:
df.groupby('Promotion').mean()['SalesInThousands'].sort_values(ascending=False)

Promotion
3    62.946686
1    61.596831
2    47.329415
Name: SalesInThousands, dtype: float64

# Answer: 3

# 7. Are the differences between strategies 1 and 2 statistically significant? Use two-sided Student T-test with confidence level 5%. (e.g. select appropriate rows in Orange, store a csv and use TTEST in google spreadsheet with 2 tails of type 2, see example in https://docs.google.com/spreadsheets/d/132BCqQUFpPbmxPvdZCFbPkkXVmx5iLHQQZas9dWi4cs/edit?usp=sharing)

In [51]:
prom_2 = df[df.Promotion == 2]['SalesInThousands']
prom_1 = df[df.Promotion == 1]['SalesInThousands']
print(np.mean(prom_1), np.mean(prom_2))

In [53]:
from scipy.stats import ttest_ind

ttest_ind(prom_1, prom_2)

Ttest_indResult(statistic=7.761907735013977, pvalue=8.85304097461179e-14)

# Answer: Yes, promotion 1 is statistically significantly better than promotion 2 with pvalue less than 1%

# 8. Are the differences between strategies 1 and 3 statistically significant? Use two-sided Student T-test with confidence level 5%. Which strategy should we select?

In [57]:
prom_3 = df[df.Promotion == 3]['SalesInThousands']
print(np.mean(prom_1), np.mean(prom_3))

61.59683139534887 62.94668617021277


In [56]:
ttest_ind(prom_1, prom_3)

Ttest_indResult(statistic=-0.6963281649703501, pvalue=0.4866751758880089)

# Answer: While there is a difference in mean sales for promotion 1 and 3, it is not statistically significant, therefore we could select any strategy.

# 9. Compare the sales for strategies 1 and 3 but now for markets of different size separatly. Compare Large markets with strategies 1 and 3. Compare Medium markets with strategies 1 and 3. Compare Small markets with strategies 1 and 3. Are the differences between strategies 1 and 3 stratified by markets of different size statistically significant (5%), which strategy should we select?

In [58]:
prom_1_large = df[(df.Promotion == 1)&(df.MarketSize == 'Large')]['SalesInThousands']
prom_3_large = df[(df.Promotion == 3)&(df.MarketSize == 'Large')]['SalesInThousands']
ttest_ind(prom_1_large, prom_3_large)

Ttest_indResult(statistic=-3.1460147617621512, pvalue=0.00216943975023725)

In [59]:
prom_1_medium = df[(df.Promotion == 1)&(df.MarketSize == 'Medium')]['SalesInThousands']
prom_3_medium = df[(df.Promotion == 3)&(df.MarketSize == 'Medium')]['SalesInThousands']
ttest_ind(prom_1_medium, prom_3_medium)

Ttest_indResult(statistic=-5.546126944655512, pvalue=8.726919451850998e-08)

In [60]:
prom_1_small = df[(df.Promotion == 1)&(df.MarketSize == 'Small')]['SalesInThousands']
prom_3_small = df[(df.Promotion == 3)&(df.MarketSize == 'Small')]['SalesInThousands']
ttest_ind(prom_1_small, prom_3_small)

Ttest_indResult(statistic=15.729103026221384, pvalue=3.286527960169143e-19)

# Answer: Run strategy 3 for Large and Medium markets and strategy 1 for Small markets

# 10. If instead of running promotions for 4 weeks we have only ran it for 1 week, does your answer to the previous question change? If we now look for confidence level 10%?

In [63]:
df_10 = df[df.week==1]

In [64]:
prom_1_large_ = df_10[(df_10.Promotion == 1)&(df_10.MarketSize == 'Large')]['SalesInThousands']
prom_3_large_ = df_10[(df_10.Promotion == 3)&(df_10.MarketSize == 'Large')]['SalesInThousands']
ttest_ind(prom_1_large_, prom_3_large_)

Ttest_indResult(statistic=-1.7931023723764705, pvalue=0.08556982681378379)

In [65]:
prom_1_medium_ = df_10[(df_10.Promotion == 1)&(df_10.MarketSize == 'Medium')]['SalesInThousands']
prom_3_medium_ = df_10[(df_10.Promotion == 3)&(df_10.MarketSize == 'Medium')]['SalesInThousands']
ttest_ind(prom_1_medium_, prom_3_medium_)

Ttest_indResult(statistic=-2.434484067162056, pvalue=0.01845222033396582)

In [66]:
prom_1_small_ = df_10[(df_10.Promotion == 1)&(df_10.MarketSize == 'Small')]['SalesInThousands']
prom_3_small_ = df_10[(df_10.Promotion == 3)&(df_10.MarketSize == 'Small')]['SalesInThousands']
ttest_ind(prom_1_small_, prom_3_small_)

Ttest_indResult(statistic=7.177589277766924, pvalue=5.208042684076091e-05)

# For 10% confidence level the answer is the same. For 5% confidence level strategy 3 is not statistically significantly different from strategy 1 for Large markets